In [2]:
import json
import os

import torch
from datasets import Dataset, load_dataset
from peft import LoraConfig
from transformers import (AutoModelForCausalLM, AutoTokenizer,
                          BitsAndBytesConfig, TrainingArguments)
from trl import SFTTrainer

os.environ["TOKENIZERS_PARALLELISM"] = "false"

In [3]:
model_path = "../models/"

data_path = "../problem_data/train.jsonl"
result_adapter = "../models/llama3-ko-alwayssaewoo_problem-adapter"


dataset = load_dataset("json", data_files=data_path, split="train")
print(dataset)

def combine(example):
    instruction = example["instruction"]
    input_text = example["input"]
    output_text = example["output"]
    example["text"] = f"<s>[INST] {instruction} {input_text} [/INST] {output_text}</s>"
    return example

dataset = dataset.map(combine)
# 토크나이저
tokenizer = AutoTokenizer.from_pretrained(model_path, use_fast=False)
tokenizer.pad_token = tokenizer.eos_token

Generating train split: 0 examples [00:00, ? examples/s]

Dataset({
    features: ['instruction', 'input', 'output'],
    num_rows: 3077
})


Map:   0%|          | 0/3077 [00:00<?, ? examples/s]

In [4]:
# 로드 Qunatization
quant_config = BitsAndBytesConfig(
    load_in_4bit = True,
    bnb_4bit_quant_type = "nf4",
    bnb_4bit_compute_dtype = torch.bfloat16
)

In [5]:
# 모델 로드
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
model = AutoModelForCausalLM.from_pretrained(
    model_path,
    quantization_config = quant_config
).to(device)

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

The module name  (originally ) is not a valid Python identifier. Please rename the original module to avoid import issues.


In [6]:
# LoRA 세팅
lora_config = LoraConfig(
    lora_alpha = 16,
    lora_dropout = 0.1,
    r = 16,
    bias = "none",
    task_type = "CAUSAL_LM"
) 

In [7]:
# 학습 arg
training_args = TrainingArguments(
    output_dir = "./arg", # 체크포인트, 로그 저장 경로
    save_total_limit = 2, # ./arg 저장 체크포인트 최대 개수
        logging_steps = 10, # 로그 출력 간격
        num_train_epochs = 3, # 에폭
        per_device_train_batch_size = 4, # GPU당 배치 크기
        gradient_accumulation_steps = 8, # gradient accumulation (미니 배치 후 업데이트)
        learning_rate = 2e-4, # 학습률
        save_steps = 50, # 모델 저장 업데이트 간격
        fp16 = True, # fp16 설정
        eval_strategy = "no" # 모델 평가 no
    )

---
테스트용


In [25]:
LoRA_tokenizer = AutoTokenizer.from_pretrained(result_adapter, use_fast=False)
LoRA_model = AutoModelForCausalLM.from_pretrained(
     model_path,
    device_map="auto",
    torch_dtype="auto"
).to(device)

`torch_dtype` is deprecated! Use `dtype` instead!


Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

The module name  (originally ) is not a valid Python identifier. Please rename the original module to avoid import issues.


In [24]:
test_data_path = "../data/test.jsonl"
test_dataset = load_dataset("json", data_files=test_data_path, split="train")

In [17]:
trainer = SFTTrainer(
    model=LoRA_model,
    args=training_args,
    train_dataset=dataset,
    eval_dataset=test_dataset,  
    peft_config=lora_config,
)

/usr/local/lib/python3.12/dist-packages/peft/mapping_func.py:72: UserWarning: You are trying to modify a model with PEFT for a second time. If you want to reload the model with a different config, make sure to call `.unload()` before.
  warnings.warn(
/usr/local/lib/python3.12/dist-packages/peft/tuners/tuners_utils.py:282: UserWarning: Already found a `peft_config` attribute in the model. This will lead to having multiple adapters in the model. Make sure to know what you are doing!
  warnings.warn(


In [18]:
eval_result = trainer.evaluate()
print(eval_result)

{'eval_loss': 2.4994373321533203, 'eval_model_preparation_time': 0.0024, 'eval_runtime': 8.4278, 'eval_samples_per_second': 12.221, 'eval_steps_per_second': 1.543, 'eval_entropy': 2.2511578523195706, 'eval_num_tokens': 0.0, 'eval_mean_token_accuracy': 0.510786411853937}


# 학습용

In [8]:
trainer = SFTTrainer(
    model=model,
    args=training_args,
    train_dataset=dataset,
    peft_config=lora_config,
)

Adding EOS to train dataset:   0%|          | 0/3077 [00:00<?, ? examples/s]

Tokenizing train dataset:   0%|          | 0/3077 [00:00<?, ? examples/s]

Truncating train dataset:   0%|          | 0/3077 [00:00<?, ? examples/s]

In [9]:
trainer.train()
trainer.save_model(result_adapter)

The tokenizer has new PAD/BOS/EOS tokens that differ from the model config and generation config. The model config and generation config were aligned accordingly, being updated with the tokenizer's values. Updated tokens: {'eos_token_id': 128009, 'pad_token_id': 128001}.


Step,Training Loss
10,2.595600
20,1.204900
30,0.849600
40,0.773400
50,0.722000
60,0.685300
70,0.673100
80,0.649700
90,0.613300
100,0.595700


In [10]:
result_adapter = "../models/llama3-ko-alwayssaewoo_problem-adapter"
trainer.save_model(result_adapter)

In [20]:
torch.cuda.empty_cache()
torch.cuda.reset_peak_memory_stats()

In [29]:
from transformers import pipeline

gen = pipeline(
    "text-generation",
    model=LoRA_model,
    tokenizer=LoRA_tokenizer,
)

for i in range(1):
    prompt = "객체지향 설계에 대해 알려줘"#test_dataset[i]["instruction"]
    print("=== 입력 ===")
    print(prompt)
    print("=== 모델 출력 ===")
    print(gen(prompt, max_new_tokens=1000)[0]["generated_text"])
    print()

Device set to use cuda:0


=== 입력 ===
객체지향 설계에 대해 알려줘
=== 모델 출력 ===
객체지향 설계에 대해 알려줘면 좋을 것 같아요. 객체지향 설계의 핵심 개념과 장점을 설명해주세요. 

객체지향 설계는 소프트웨어 개발에서 매우 중요한 역할을 합니다. 객체지향 설계의 핵심 개념과 장점을 설명드리겠습니다.

### 객체지향 설계의 핵심 개념

1. **객체(Object)**: 객체는 실세계의 사물이나 개념을 코드로 표현한 것입니다. 객체는 속성(데이터)과 메서드(기능)를 가지고 있습니다.

2. **클래스(Class)**: 클래스는 객체를 생성하기 위한 청사진입니다. 클래스는 객체의 속성과 메서드를 정의합니다.

3. **상속(Inheritance)**: 상속은 기존 클래스의 속성과 메서드를 물려받아 새로운 클래스를 만들 수 있게 해줍니다. 이를 통해 코드의 재사용성을 높일 수 있습니다.

4. **다형성(Polymorphism)**: 다형성은 동일한 인터페이스나 메서드를 통해 다양한 객체를 다룰 수 있게 해줍니다. 이는 코드의 유연성과 확장성을 높입니다.

5. **캡슐화(Encapsulation)**: 캡슐화는 객체의 속성을 외부로부터 숨기고, 필요한 경우에만 공개하는 것을 의미합니다. 이는 데이터의 보호와 코드의 유지보수를 용이하게 합니다.

6. **Composition**: 구성 관계는 하나의 객체가 다른 객체의 집합인 경우를 의미합니다. 이는 복잡한 객체를 단순화하고, 코드의 재사용성을 높입니다.

### 객체지향 설계의 장점

1. **모듈화**: 객체지향 설계는 소프트웨어를 작은 모듈로 나누어 관리할 수 있게 해줍니다. 각 모듈은 독립적으로 개발되고 테스트될 수 있습니다.

2. **재사용성**: 재사용 가능한 코드를 작성할 수 있습니다. 이미 작성된 클래스나 메서드를 다른 프로젝트에서도 사용할 수 있습니다.

3. **유지보수성**: 코드의 유지보수가 용이합니다. 객체의 변경은 해당 객체만을 변경하면 되므로, 전체 시스템에 미치는 영향이 적습니다.

4. *